In [1]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('../aRDee/mydata/training_set')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['J', 'Z', 'K', 'Q', 'I', 'N', 'O', 'G', 'D', 'C', 'Y', 'U', 'T', 'H', 'S', 'A', 'F', 'W', 'V', 'E', 'P', 'L', 'B', 'R', 'X', 'M']
Types of classes labels found:  26


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0 (weights = 'imagenet')

In [2]:
class_labels = []
path = r"../aRDee/mydata/training_set"

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir(path + '/' +item)
#  print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

In [3]:
# Build a dataframe        
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

  Labels                    image
0      J   dataset_path/J/913.png
1      J   dataset_path/J/174.png
2      J   dataset_path/J/239.png
3      J  dataset_path/J/1267.png
4      J   dataset_path/J/726.png
      Labels                    image
45495      M     dataset_path/M/8.png
45496      M  dataset_path/M/1706.png
45497      M  dataset_path/M/1018.png
45498      M   dataset_path/M/825.png
45499      M   dataset_path/M/992.png


In [4]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  45500
J    1750
Z    1750
X    1750
R    1750
B    1750
L    1750
P    1750
E    1750
V    1750
W    1750
F    1750
A    1750
S    1750
H    1750
T    1750
U    1750
Y    1750
C    1750
D    1750
G    1750
O    1750
N    1750
I    1750
Q    1750
K    1750
M    1750
Name: Labels, dtype: int64


In [5]:
# Resize images

import cv2
path = "../aRDee/mydata/training_set/"
dataset_path = os.listdir("../aRDee/mydata/training_set/")

im_size = 96

images = []
labels = []

for i in dataset_path:
    print(i)
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)


J
Z
K
Q
I
N
O
G
D
C
Y
U
T
H
S
A
F
W
V
E
P
L
B
R
X
M


In [6]:
#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255]. 

images = np.array(images)
labels = np.array(labels)
images = images.astype('float32') / 255.0
images.shape

(45500, 96, 96, 3)

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['J' 'J' 'J' ... 'M' 'M' 'M']
[ 9  9  9 ... 12 12 12]


In [8]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, 26)

In [9]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.2)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(36400, 96, 96, 3)
(36400, 26)
(9100, 96, 96, 3)
(9100, 26)


In [11]:
# from keras.preprocessing.image import ImageDataGenerator

# train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )
# test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

# train_generator.fit(train_x)
# test_generator.fit(test_x)

# #Learning Rate Annealer
# from keras.callbacks import ReduceLROnPlateau
# lrr= ReduceLROnPlateau( monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)

In [ ]:
from keras import Sequential
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 26
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

outputs = EfficientNetB0(include_top = True, weights = None, classes = NUM_CLASSES) (inputs)

In [1]:
IMG_SIZE = 224
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(IMG_SIZE,IMG_SIZE,3),classes=26)


NameError: name 'EfficientNetB0' is not defined

In [13]:
model= Sequential()
model.add(base_model) 
model.add(Flatten()) 

#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=512))

model.add(Dense(512,activation=('relu'))) 
# model.add(Dense(256,activation=('relu'))) 
# #model.add(Dropout(.3))
# model.add(Dense(128,activation=('relu')))
# #model.add(Dropout(.2))
# model.add(Dense(29,activation=('softmax'))) 

#Checking the final model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb5 (Functional)  (None, 7, 7, 2048)       28513527  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
Total params: 131,799,799
Trainable params: 131,627,056
Non-trainable params: 172,743
_________________________________________________________________


In [14]:
learn_rate=.001
from tensorflow.keras.optimizers import SGD
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

/home/sukanya/.local/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau( monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)


#Defining the parameters
batch_size= 32
epochs=2
learn_rate=.001
model.fit(train_x, train_y, batch_size = batch_size, epochs = epochs, steps_per_epoch = train_x.shape[0]//batch_size)